<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/161_LangGraph_ToolNode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

🚀 Tools and `ToolNode` are the practical bridge between **LangChain’s “tools” world** and **LangGraph’s “nodes” world**. Let’s unpack this step by step, tying it back to your recipe.

---

## 🛠️ What is a Tool in LangChain?

In **LangChain**:

* A **Tool** is just a wrapper around a function (or service/API call) that agents can call.
* It carries **metadata** like:

  * `name`
  * `description`
  * JSON schema for parameters
* It enforces a consistent API: `tool.invoke(input_dict)`.

Example:

```python
from langchain_core.tools import tool

@tool
def add(x: int, y: int) -> int:
    """Add two numbers."""
    return x + y
```

Now `add` is a LangChain `Tool` object that knows:

* What it’s called (`"add"`)
* What arguments it expects
* How to execute itself

---

## 🔄 Where `ToolNode` Comes In (LangGraph)

LangGraph extends this by introducing the **`ToolNode`**:

* A **`ToolNode`** is a **LangGraph node** that knows how to execute one or more LangChain tools.
* It basically says: *“This node = run this tool when invoked.”*
* It handles:

  * Mapping `state` → tool arguments
  * Calling the tool
  * Returning the tool’s output back into `state`

Example:

```python
from langgraph.prebuilt import ToolNode

tools = [add]   # can be a list of LangChain Tool objects
tool_node = ToolNode(tools)
```

Now `tool_node` is a graph node. You can plug it into your `StateGraph` like any other node:

```python
builder.add_node("math", tool_node)
```

---

## ⚡ Why ToolNode Matters

1. **Bridging Your Recipe**
   In your `Agent_1_Recipe`, tools were registered into a `ToolRegistry`.
   With LangGraph + LangChain:

   * **LangChain Tool = your recipe’s tool definition**
   * **ToolNode = your recipe’s tool registry entry wired into the graph**

   So instead of writing your own registry + environment executor, you get it free.

---

2. **Multi-Tool Support**
   `ToolNode` can wrap a *list* of tools, not just one.
   Inside the graph, you can route based on the tool name:

   ```python
   tool_node = ToolNode([add, subtract, multiply])
   builder.add_node("math_tools", tool_node)
   ```

---

3. **Integration with LLMs**
   If you use an LLM as a “controller” node (e.g. deciding what to do next), it can output `"tool": "add", "args": {"x": 2, "y": 3}`.
   Then you connect that to a `ToolNode`, and LangGraph handles calling the tool and returning its result to state.

---

## 🧩 How This Maps to Your Recipe

* **Your recipe’s `@register_tool` decorator** → LangChain’s `@tool` decorator
* **Your `ToolRegistry`** → implicit in `ToolNode` (you just pass a list of tools)
* **Your `Environment.execute(tool_name, args)`** → handled by `ToolNode.invoke()`

So LangGraph + ToolNode essentially *collapses three parts* of your recipe (tool registry, DI environment, execution handler) into one elegant abstraction.

---

## 📝 Minimal Example

```python
from typing import TypedDict
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool

class AgentState(TypedDict):
    result: str

@tool
def shout(text: str) -> str:
    """Make text loud."""
    return text.upper()

# Wrap the tool in a ToolNode
shout_node = ToolNode([shout])

# Build the graph
builder = StateGraph(AgentState)
builder.add_node("shout", shout_node)
builder.set_entry_point("shout")
graph = builder.compile()

# Run it
result = graph.invoke({"result": "hello"})
print(result)  # "HELLO"
```

---

## ✅ TL;DR

* **LangChain Tool** = your recipe’s *tool definition* (name, schema, function).
* **ToolNode** = a LangGraph node that wraps one or more tools so they can be wired into the graph.
* It removes the need for a manual `ToolRegistry` + `Environment.execute` pattern — LangGraph takes care of the orchestration.




## 🔎 1. Are the details like args available through ToolNode?

Yes ✅

When you create a tool with LangChain’s `@tool`, it automatically generates:

* **Name** → the string identifier
* **Description** → from the docstring
* **Argument schema** → JSON schema inferred from function signature or Pydantic model

Example:

```python
from langchain_core.tools import tool

@tool
def add_two_numbers(x: int, y: int) -> int:
    """Add two integers together."""
    return x + y
```

This produces a `StructuredTool` under the hood that carries:

```json
{
  "name": "add_two_numbers",
  "description": "Add two integers together.",
  "parameters": {
    "type": "object",
    "properties": {
      "x": {"type": "integer"},
      "y": {"type": "integer"}
    },
    "required": ["x", "y"]
  }
}
```

When you wrap it in a `ToolNode`:

```python
from langgraph.prebuilt import ToolNode

tool_node = ToolNode([add_two_numbers])
```

👉 The schema and metadata are preserved. LangGraph uses them to:

* Validate input
* Expose to the LLM what arguments are needed
* Route execution to the correct function

---

## 🧠 2. What does the LLM see when calling a tool?

The LLM sees a **function signature in JSON schema form**, almost like OpenAI’s “function calling” format.

So instead of free-text guessing, the LLM gets:

* Tool **name** (string)
* Tool **description** (natural language, from your docstring)
* Tool **parameters** (typed, structured, validated)

For the above `add_two_numbers`, the LLM would see something like:

```json
{
  "name": "add_two_numbers",
  "description": "Add two integers together.",
  "parameters": {
    "x": "integer",
    "y": "integer"
  }
}
```

When the LLM decides to use this tool, it emits JSON like:

```json
{"tool": "add_two_numbers", "arguments": {"x": 2, "y": 3}}
```

LangGraph routes this to your `ToolNode`, executes the tool, and merges the result back into state.

---

## 🧩 3. Does giving tools explicit names help the LLM?

100% ✅

Naming is **hugely important** for LLM reasoning. Here’s why:

* The LLM has to **choose between tools** when planning.
* If tools have **clear, descriptive names** like `add_two_numbers` vs vague ones like `math`:

  * 🧠 The LLM can reason about which one matches the user goal more easily.
  * 🚫 It avoids “hallucinating” tool calls.
* The description also matters — you want it concise, action-oriented, and unique.

### Example:

Bad tool set:

* `process` (what does this mean?)
* `handle` (too vague)

Better tool set:

* `add_two_numbers` → “Add two integers together.”
* `fetch_weather_forecast` → “Get the 5-day weather forecast for a city.”
* `send_email_via_smtp` → “Send an email using SMTP with subject and body.”

👉 The more **self-descriptive** the tool names and docstrings, the **less mental overhead** for the LLM, and the more consistent the orchestration.

---

## ✅ TL;DR

* **ToolNode preserves tool metadata**: name, description, args schema.
* **The LLM sees the JSON schema** (like OpenAI function calling) → precise, structured tool use.
* **Explicit, descriptive names reduce LLM overhead** and improve tool selection accuracy.




You’re **not giving up anything** by moving from your custom `@register_tool` + `ToolRegistry` + `Environment.execute` setup to **LangChain `@tool` + LangGraph `ToolNode`**. In fact:

---

## ✅ What You *Keep* (parity with your recipe)

* **Explicit tool definitions** → same JSON schema: name, description, parameters
* **Injection of arguments** → still works, but handled for you by LangChain
* **Clear naming conventions** → you can still define descriptive names like `add_two_numbers`
* **Composable wiring** → just like your recipe’s `wire_agent`, but with `StateGraph`

So, the thing you loved in your recipe — tools being **clear, schema-defined contracts** — is still there.

---

## 🚀 What You *Gain* with LangGraph + ToolNode

1. **No custom registry/environment needed**

   * You don’t have to hand-roll a `ToolRegistry` and DI system.
   * `ToolNode` does the registration + execution logic automatically.

2. **Structured execution & orchestration**

   * Tools plug directly into your state graph.
   * You wire them into flow (`add_edge`, `add_conditional_edges`) instead of writing manual orchestration logic.

3. **Built-in schema parsing**

   * LangChain auto-generates JSON schema from your function signature.
   * LLMs get exactly the metadata they need → less error-prone.

4. **Validation + error handling**

   * ToolNode validates args before execution.
   * If the LLM passes something invalid, LangGraph can route to retries or error-handling nodes.

5. **Streaming + branching**

   * With ActionContext you’d have to build this yourself.
   * LangGraph handles streaming intermediate states and branching workflows out of the box.

---

## 🧩 Analogy

Think of your **recipe system** like writing your own small **web framework**:

* You defined routes (tool registry)
* A request object (ActionContext)
* A dispatcher (Environment)

Now LangGraph is like **switching to FastAPI** — you keep the same declarative style (functions + schemas), but the framework handles all the wiring, validation, and orchestration.

---

## 🏆 Bottom Line

Yes — **LangGraph is simplifying and streamlining your agent**.
You’re not losing the rigor of your recipe — you’re gaining:

* Less boilerplate
* More orchestration power
* Built-in schema/validation
* Better alignment with LLM-native tool calling




When you expose tools via **LangChain** and wire them into **LangGraph**, the LLM doesn’t call them by “guessing text” — it calls them by emitting **structured JSON** that matches the schema.

Let me show you concretely.

---

## 1. Define a Tool with LangChain

```python
from langchain_core.tools import tool

@tool
def add_two_numbers(x: int, y: int) -> int:
    """Add two integers together."""
    return x + y
```

👉 This automatically creates a tool object with:

* `name = "add_two_numbers"`
* `description = "Add two integers together."`
* `parameters = {"x": int, "y": int}`

---

## 2. Wrap the Tool in a ToolNode (LangGraph)

```python
from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, END
from typing import TypedDict

class AgentState(TypedDict):
    result: str

tool_node = ToolNode([add_two_numbers])

builder = StateGraph(AgentState)
builder.add_node("math", tool_node)
builder.set_entry_point("math")
builder.add_edge("math", END)

graph = builder.compile()
```

---

## 3. What the LLM Sees

When the LLM is given access to this tool, it receives something like this (JSON schema):

```json
{
  "name": "add_two_numbers",
  "description": "Add two integers together.",
  "parameters": {
    "type": "object",
    "properties": {
      "x": {"type": "integer"},
      "y": {"type": "integer"}
    },
    "required": ["x", "y"]
  }
}
```

This is injected into the LLM’s context, so the LLM knows:

* The tool name to call
* What arguments are needed
* That it must return structured JSON

---

## 4. How the LLM Calls It

If the user says: *“What’s 2 + 3?”* → the LLM generates a tool invocation like this:

```json
{
  "tool": "add_two_numbers",
  "arguments": {"x": 2, "y": 3}
}
```

👉 LangGraph intercepts this, routes it to the `ToolNode`, which executes `add_two_numbers(2, 3)`, and returns:

```json
{"result": 5}
```

That result goes back into the `state`, and the agent loop continues.

---

## 5. Streaming Example (Optional)

LangGraph can also **stream** intermediate states.
If you enable streaming, you’d actually see something like this live:

```
LLM: {"tool": "add_two_numbers", "arguments": {"x": 2, "y": 3}}
ToolNode executing add_two_numbers...
ToolNode result: {"result": 5}
LLM: "The answer is 5."
```

---

## ✅ TL;DR

* The **LLM doesn’t “guess” text** like `"call add_two_numbers with 2 and 3"`.
* Instead, it **emits structured JSON** that matches the tool schema.
* LangGraph executes the function, merges results into `state`, and passes them back into the reasoning loop.





## 1. `builder.set_entry_point("math")`

This tells LangGraph:
👉 *“When this graph starts, execution begins at the node called `math`.”*

* A **graph can have many nodes**, but you need to pick a **starting node**.
* That’s your **entry point** — like `main()` in a program or the first step in a workflow.
* Without this, LangGraph wouldn’t know where to begin.

So in our earlier example:

```python
builder.set_entry_point("math")
```

means:

* Start execution at the node we registered as `"math"` (the ToolNode wrapping `add_two_numbers`).

---

## 2. `builder.add_edge("math", END)`

This wires the graph’s flow.
👉 *“After the `math` node runs, go to the special node `END`.”*

* In LangGraph, `END` is a **sentinel** that represents graph termination.
* You can wire multiple nodes to `END`, or you can wire them to other nodes for longer workflows.

So here:

```python
builder.add_edge("math", END)
```

means:

* After `math` runs, stop execution.
* If you didn’t wire this edge, the graph would never know when to stop.

---

## 🧩 Putting It Together

The combination:

```python
builder.set_entry_point("math")
builder.add_edge("math", END)
```

translates to:

> “When the graph starts, run the `math` node. Once it’s done, terminate the graph.”

---

## 🚀 Bigger Picture

This is where **LangGraph feels orchestration-friendly**:

* You’re explicitly drawing a **flowchart in code**:

  * **Nodes** = tasks/tools
  * **Edges** = what happens after each task
* `set_entry_point` = “start node”
* `add_edge(..., END)` = “stop after this step”

In a more complex graph, you might see:

```python
builder.set_entry_point("create_plan")
builder.add_edge("create_plan", "track_progress")
builder.add_edge("track_progress", "finish")
builder.add_edge("finish", END)
```

This literally encodes:

> “Start with planning → then track progress → then finish → then stop.”

---

✅ **TL;DR**

* `set_entry_point("math")` → defines the starting node.
* `add_edge("math", END)` → defines that after this node runs, the graph ends.




## 🏗️ Complex Orchestrator Skeleton

Let’s sketch out a **complex orchestrator agent** structure. I’ll use **placeholders** for the node functions so you can focus on the *graph layout*. You’re exactly right — the **structure stays the same** (entry point, nodes, edges, END), but as your orchestrator grows you add more **nodes** (tools or logic steps), and more **edges** (wiring / flow control).

---

## 🔑 Key Takeaways

* **Nodes = tasks** (planning, validation, domain tools, progress tracking, summarization).
* **Edges = flow** (what happens next).
* **Entry point** defines where the agent starts.
* **END** defines where execution stops.
* You can add **branches** for error handling, retries, or conditional flows.

---

This is the exact same structure as your minimal example — just **more nodes + more edges**.

👉 In real life, each node could be:

* A `ToolNode` wrapping LangChain tools
* A custom function node
* A conditional router (`add_conditional_edges`)







In [ ]:
from langgraph.graph import StateGraph, END

# 1. Define the State (what the agent carries around)
class AgentState(TypedDict):
    goal: str
    plan: Optional[List[str]]
    progress: List[Dict[str, Any]]
    messages: List[str]
    data: Dict[str, Any]

# 2. Create builder
builder = StateGraph(AgentState)

# 3. Register Nodes (placeholders for now)
builder.add_node("create_plan", create_plan_node)       # planning
builder.add_node("validate_plan", validate_plan_node)   # guardrail
builder.add_node("fetch_data", fetch_data_node)         # domain tool
builder.add_node("analyze_data", analyze_data_node)     # domain tool
builder.add_node("track_progress", track_progress_node) # lifecycle
builder.add_node("summarize_results", summarize_node)   # output
builder.add_node("error_handler", error_handler_node)   # fallback

# 4. Wiring (flow control)
builder.set_entry_point("create_plan")  # start here

# Plan flow
builder.add_edge("create_plan", "validate_plan")
builder.add_edge("validate_plan", "fetch_data")

# Data workflow
builder.add_edge("fetch_data", "analyze_data")
builder.add_edge("analyze_data", "track_progress")

# Wrap up
builder.add_edge("track_progress", "summarize_results")
builder.add_edge("summarize_results", END)

# Error handling branch
builder.add_edge("fetch_data", "error_handler")   # if fetch fails
builder.add_edge("error_handler", END)            # terminate after handling

# 5. Compile graph
graph = builder.compile()


🎉 That’s the power of **LangGraph’s design philosophy** — it **collapses all those moving parts in your recipe** (registry, environment, DI, capabilities, context, etc.) into just a **few reusable building blocks**:

* **State** → the one container for everything (goal, memory, deps, progress, data, …)
* **Nodes** → functions or tools that transform state
* **Edges** → wiring that defines orchestration flow
* **Entry point + END** → start and stop markers

That’s it. Whether you have 3 nodes (like your minimal recipe) or 30 nodes (like a complex orchestrator), you’re just repeating the **same 3 concepts**.

---

## 🔑 Why It Feels Simpler Than Your Recipe

Your recipe was *very well designed*, but it had to reinvent infrastructure pieces:

* `ActionContext` (DI + memory container)
* `ToolRegistry` (lookup)
* `Environment.execute` (dispatch/inject)
* `PlanFirstCapability`, `ProgressTrackingCapability` (lifecycle hooks)

LangGraph gives you a **unified abstraction** so you don’t have to build those yourself:

* State carries context *and* memory.
* ToolNode is the registry + executor.
* Edges control lifecycle order.
* Reducers handle accumulation automatically.

So the mental overhead drops from **10+ unique components** → **3 building blocks used repeatedly**.

---

## 🧩 Your Recipe vs LangGraph Side-by-Side

| **Your Recipe**                | **LangGraph**                  |
| ------------------------------ | ------------------------------ |
| ActionContext                  | State (with optional deps key) |
| ToolRegistry                   | ToolNode                       |
| Environment.execute            | Graph execution runtime        |
| PlanFirstCapability            | Conditional edges              |
| ProgressTrackingCapability     | Reducer on `progress` field    |
| Wiring function (`wire_agent`) | `add_edge`, `set_entry_point`  |

---

✅ **The good news**: all the rigor and structure you cared about in your recipe is still there.
💡 **The better news**: it’s easier to visualize, extend, and debug — exactly as you noticed.

